In [1]:
import pandas as pd
import import_ipynb
import h1_ENB_occ as h1 
import h2_ENB_gen as h2 

importing Jupyter notebook from h1_ENB_occ.ipynb
importing Jupyter notebook from c1_extract_paragraphe_issue.ipynb
Issue  34
importing Jupyter notebook from c2_extract_sentence_issue.ipynb
importing Jupyter notebook from g1_generate_dictionary.ipynb
importing Jupyter notebook from h2_ENB_gen.ipynb
[nltk_data] Downloading package punkt to /Users/tes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tes/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
['COMIFAC ', 'PERU', 'CACAM', 'ITALY', 'HAITI', 'MOUNTAIN LANDLOCKED DEVELOPING COUNTRIES', 'PANAMA', 'CZECH REPUBLIC', 'ERITREA', 'ANTINGUA AND BARBUDA', 'ANTIGUA AND BARBUDA', 'MAURITANIA', 'NIUE', 'YEMEN', 'TOGO', 'AZERBAIJAN', 'SRI LANKA', 'MALAYSIA', 'ECUADOR', 'COMOROS', 'OPEC', 'for the Arab Group', 'for the League of Arab States', 'ARAB GROUP', 'CAMBODIA', 'UKRAINE', 'EQUATORIAL GU

In [32]:
def extract_list_issue():
     with open("Text/ENB_relationships.txt") as infile:
        k=0
        num = []
        for line in infile:
            x = line.replace('"','').split('\t')[:26]

            if(k > 0 and int(x[23]) < 594  and 100 < int(x[23])):

                num.append(int(x[23]))
            k +=1
        return list(set(num))

In [33]:
list_issue_num = extract_list_issue()

In [34]:
def extract_gen():
    frames1 = []
    frames2 = []
    for issue_number in list_issue_num:
        df1 = h1.extract_relationships_count(issue_number)
        df2 = h2.interactions(issue_number)
        frames1.append(df1)
        frames2.append(df2) 
    return pd.concat(frames1), pd.concat(frames2)


In [35]:
df_original, df_generated = extract_gen()

Issue  101
Issue  102
Issue  103
Issue  104
Issue  105
Issue  106
Issue  107
Issue  108
Issue  109
Issue  113
Issue  114
Issue  115
Issue  116
Issue  117
Issue  118
'FEDERATED STATES OF MICRONESIA'
'BAHAMAS'
'CHILE'
'EU'
'FEDERATED STATES OF MICRONESIA'
'AUSTRALIA'
'US'
'FEDERATED STATES OF MICRONESIA'
'FEDERATED STATES OF MICRONESIA'
'FEDERATED STATES OF MICRONESIA'
'FEDERATED STATES OF MICRONESIA'
'FEDERATED STATES OF MICRONESIA'
'NEW ZEALAND'
'ARGENTINA'
Issue  119
Issue  120
'UK'
'CARICOM'
'IRELAND'
'CARICOM'
'FRANCE'
'SPAIN'
'GERMANY'
'CARICOM'
'EU'
'CARICOM'
'CARICOM'
'CARICOM'
'ITALY'
'SWEDEN'
'CARICOM'
'CARICOM'
'BULGARIA'
'HAITI'
'CARICOM'
'JAPAN'
'CARICOM'
'CARICOM'
Issue  121
Issue  124
Issue  125
Issue  126
Issue  128
Issue  129
Issue  130
Issue  131
Issue  132
Issue  133
Issue  134
Issue  135
Issue  136
Issue  138
Issue  139
Issue  142
Issue  143
Issue  144
Issue  145
Issue  146
'FEDERATED STATES OF MICRONESIA'
'FEDERATED STATES OF MICRONESIA'
'SAMOA'
'FEDERATED STATES OF 

In [47]:
df_original
#df_generated

,type,issue,id_ca,id_cb,Country A,Country B,behalf,support,agreement,opposition,criticism,cooperation
0,original,101,205,69,AFRICAN GROUP,NORWAY,0,0,1,0,0,1
1,original,101,69,205,NORWAY,AFRICAN GROUP,0,0,1,0,0,1
2,original,101,130,197,UNITED STATES,JAPAN,0,0,0,0,0,1
3,original,101,130,53,UNITED STATES,GUATEMALA,0,0,0,0,0,1
4,original,101,197,130,JAPAN,UNITED STATES,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
63,original,593,86,197,MARSHALL ISLANDS,JAPAN,0,0,0,1,0,0
64,original,593,86,201,MARSHALL ISLANDS,AUSTRALIA,0,0,0,1,0,0
65,original,593,86,50,MARSHALL ISLANDS,LIKE MINDED DEVELOPING COUNTRIES,0,0,0,1,0,0
66,original,593,29,99,INDONESIA,NAURU,0,1,0,0,0,1


In [45]:
df = pd.concat([df_generated, df_original])
df = df[['issue', 'Country A', 'Country B', 'id_ca', 'id_cb','cooperation','agreement','behalf','criticism','opposition','support','type']]
d = df.drop(['Country A','Country B','type','issue'],axis=1)
#d = df.drop(['Country A','Country B','type','agreement','behalf','criticism','opposition','support'],axis=1)

d = d.drop_duplicates(keep = False)

d.sort_values(by=['id_ca','id_cb'], inplace=True)

In [46]:
d

,id_ca,id_cb,cooperation,agreement,behalf,criticism,opposition,support
24,0,5,1.0,1.0,0.0,0.0,0.0,0.0
16,0,39,1.0,0.0,0.0,0.0,0.0,0.0
26,0,45,1.0,1.0,0.0,0.0,0.0,0.0
31,0,65,0.0,0.0,0.0,0.0,1.0,0.0
66,0,69,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
31,"align=""justify""&gt;The EU, also speaking for t...",210,1.0,1.0,0.0,0.0,0.0,0.0
11,"class=""textstory""&gt;Kyrgyzstan, on behalf of ...",5,1.0,1.0,0.0,0.0,0.0,0.0
12,"class=""textstory""&gt;Kyrgyzstan, on behalf of ...",124,1.0,1.0,0.0,0.0,0.0,0.0
2,"class=""textstory""&gt;Samoa, speaking for the C...",6,1.0,1.0,0.0,0.0,0.0,0.0


missing id dict

problems

' Malaysia and Ghana, both speaking for the G-77/CHINA, suggested some additions.'Issue  283'

# Only cooperation
    With Issue : 2185 rows × 4 columns
    Without Issue : 835 rows × 3 columns

# With all of them 
    With Issue : 1183 rows × 8 columns
    Without Issue : 2794 rows × 9 columns

In [ ]:
# 100 - 594
Total number of observations
Generated : 32225
Original : 44572
Only cooperation 
    without issue : 3714 rows × 3 columns
    with issue : 18230
All interactions
    without issue : 6265
    with issue : 26326